In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
tf.test.is_gpu_available()
gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=14336 )] #7168  6144
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

In [ ]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from gensim.models import Word2Vec
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
import warnings
warnings.filterwarnings('ignore') 
#from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import nltk
nltk.download('punkt')
from spacy.lang.tr import Turkish
from tqdm import tqdm
import string
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline
import time
from keras.utils import np_utils

In [ ]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_preprocessing.xlsx")

#column = ['tweets','duygu']
#df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
#df = pd.read_excel("../dataset/preprocessing_kemik.xlsx")

df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [ ]:
X_t=df['tweets'].tolist()

In [ ]:
# Importing library
from keras.preprocessing.text import Tokenizer
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [ ]:
#!pip install glove-python-binary

In [ ]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(X_t, window=5)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=256, learning_rate=0.1)
 
glove.fit(corpus.matrix, epochs=200, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [ ]:
wordembeddings=glove.load('glove.model')

In [ ]:
def glove (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi
    y_train = np_utils.to_categorical(y_train)

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)
    y_test = np_utils.to_categorical(y_test)

    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 256 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(256,activation='relu',return_sequences= True))
    model.add(SimpleRNN(128,activation='relu'))
    model.add(Dense(3, activation='softmax'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = model.evaluate(X_test, y_test)
    preds = model.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#cnn

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn

    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=128, kernel_size=3, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=3)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(64, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(16, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(3, activation='softmax'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelCNN.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelCNN.evaluate(X_test, y_test)
    preds = modelCNN.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)

    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(3, activation='softmax'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelLSTM.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelLSTM.evaluate(X_test, y_test)
    preds = modelLSTM.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#cnn+lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)

    modelCNNLSTM = Sequential() #cnn

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=128, kernel_size=3, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNNLSTM.add(MaxPooling1D(pool_size=3)) #tek satırlık 1d olduğu için
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(Dense(64, activation='relu'))
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(Dense(16, activation='relu'))
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(LSTM(128))
    modelCNNLSTM.add(Flatten()) #düzgünleştirmek için
    modelCNNLSTM.add(Dense(3, activation='softmax'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelCNNLSTM.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelCNNLSTM.evaluate(X_test, y_test)
    preds = modelCNNLSTM.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))